In [6]:
# 聊天机器人教程
# 同时可以参考： https://www.cntofu.com/book/169/docs/1.0/chatbot_tutorial.md

# 在本教程中，我们探索了一个好玩和有趣的循环序列到序列的模型用例。
# 我们将用 Cornell Movie-Dialogs Corpus处的电影剧本来训练一个简单的聊天机器人。

# 在人工智能研究领域中对话模型是一个非常热门的话题。聊天机器人可以在各种设置中找到，
# 包括客户服务应用和在线帮助。这些机器人通常由基于检索的模型提供支持，这些输出是某些形式问题预先定义的响应。
# 在像公司IT服务台这样高度受限制的领域中，这些模型可能足够了，但是，对于更一般的用例它们不够健壮。
# 教一台机器与多领域的人进行有意义的对话是一个远未解决的研究问题。最近，深度学习热潮已经允许强大的生成模型，
# 如谷歌的神经对话模型 Neural Conversational Model(https://arxiv.org/abs/1506.05869)，这标志着向多领域生成对话模型迈出了一大步。 
# 在本教程中，我们将在PyTorch中实现这种模型

# 教程要点
# 1 对Cornell Movie-Dialogs Corpus 数据集的加载和预处理
# 2 用 Luong attention mechanism(s)实现一个sequence-to-sequence模型
# 3 使用小批量数据联合训练解码器和编码器模型
# 4 实现贪婪搜索解码模块
# 5 与训练好的聊天机器人互动

# 本教程借用以下来源的代码：

# Yuan-Kuei Wu’s pytorch-chatbot implementation: 
#            https://github.com/ywk991112/pytorch-chatbot
# Sean Robertson’s practical-pytorch seq2seq-translation example: 
#            https://github.com/spro/practical-pytorch/tree/master/seq2seq-translation
# FloydHub’s Cornell Movie Corpus preprocessing code: 
#            https://github.com/floydhub/textutil-preprocess-cornell-movie-corpus

In [7]:
# 准备工作

# 首先，下载数据文件here(https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)
# 并将其放入当前目录下的data/文件夹下。
# 之后，让我们引入一些必须的包。

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [8]:
# 加载和预处理数据

# 下一步就是格式化处理我们的数据文件并加载到我们可以使用的结构中
# Cornell Movie-Dialogs Corpus 是一个丰富的电影角色对话数据集：
#    * 10,292 对电影角色的220,579 次对话
#    * 617部电影中的9,035电影角色
#    * 总共304,713中语调
# 这个数据集庞大而多样，在语言形式、时间段、情感上等都有很大的变化。我们希望这种多样性使我们的模型能够适应多种形式的输入和查询。

# 首先，我们通过数据文件的某些行来查看原始数据的格式

corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [9]:
# 创建格式化数据文件

# 为了方便起见，我们将创建一个格式良好的数据文件，其中每一行包含一个由tab制表符分隔的查询语句和响应语句对。

# 以下函数用于解析原始数据文件raw movie_lines.txt
#   * loadLines 将文件的每一行拆分为字段(lineID, characterID, movieID, character, text)组合的字典
#   * loadConversations 根据movie_conversations.txt将loadLines 中的每一行数据进行归类
#   * extractSentencePairs 从对话中提取一对句子

# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields like ["lineID", "characterID", "movieID", "character", "text"]
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines

# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields like ["character1ID", "character2ID", "movieID", "utteranceIDs"]
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            # liujia: eval()对字符串形式的"['L271', 'L272', 'L273', 'L274', 'L275']"，转换为list。。。。
            lineIds = eval(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations

# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        # liujia: 下面这一行已经去掉了最后一行了，实际上range返回的是第一行到倒数第二行的序号
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs


# 现在我们将调用这些函数来创建文件，我们命名为 formatted_movie_lines.txt.

# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

# Unescape the delimiter
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape")) # unicode-escape也是一种编码集，类似utf-8，是将unicode内存编码值直接存储

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)

print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    # liujia: 调用csv库的函数，写成标准的csv文件格式
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)



Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't dat

In [10]:
# 加载和清洗数据

# 我们下一个任务是创建词汇表并将查询/响应句子对（对话）加载到内存。
# 注意我们正在处理词序，这些词序没有映射到离散数值空间。因此，我们必须通过数据集中的单词来创建一个索引。
# 为此我们创建了一个Voc类,它会存储从单词到索引的映射、索引到单词的反向映射、每个单词的计数和总单词量。
# 这个类提供向词汇表中添加单词的方法(addWord)、添加所有单词到句子中的方法 (addSentence) 和清洗不常见的单词方法(trim)。
# 更多的数据清洗在后面进行。

# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

# 现在我们可以组装词汇表和查询/响应语句对。在使用数据之前，我们必须做一些预处理。
# 首先，我们必须使用unicodeToAscii将unicode字符串转换为ASCII。
# 然后，我们应该将所有字母转换为小写字母并清洗掉除基本标点之外的所有非字母字符 (normalizeString)。
# 最后，为了帮助训练收敛，我们将过滤掉长度大于MAX_LENGTH 的句子 (filterPairs)。

MAX_LENGTH = 15  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to: https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    print('corpus:', corpus)
    print('corpus_name:', corpus_name)
    print('datafile:', datafile)
    print('save_dir:', save_dir)
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)
    

    
# 另一种有利于让训练更快收敛的策略是去除词汇表中很少使用的单词。减少特征空间也会降低模型学习目标函数的难度。
# 我们通过以下两个步骤完成这个操作:
#    * 使用voc.trim 函数去除 MIN_COUNT阈值以下单词 。
#    * 如果句子中包含词频过小的单词，那么整个句子也被过滤掉。

MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs

# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

Start preparing training data ...
corpus: data/cornell movie-dialogs corpus
corpus_name: cornell movie-dialogs corpus
datafile: data/cornell movie-dialogs corpus/formatted_movie_lines.txt
save_dir: data/save
Reading lines...
Read 221282 sentence pairs
Trimmed to 111344 sentence pairs
Counting words...
Counted words: 26856

pairs:
['no no it s my fault we didn t have a proper introduction', 'cameron .']
['gosh if only we could find kat a boyfriend . . .', 'let me see what i can do .']
['c esc ma tete . this is my head', 'right . see ? you re ready for the quiz .']
['that s because it s such a nice one .', 'forget french .']
['how is our little find the wench a date plan progressing ?', 'well there s someone i think might be']
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
keep_words 13150 / 26853 = 0.4897
Tr

In [11]:
# 为模型准备数据

# 尽管我们已经投入了大量精力来准备和清洗我们的数据变成一个很好的词汇对象和一系列的句子对，
# 但我们的模型最终希望以numerical torch张量作为输入。可以在seq2seq translation tutorial中找到为模型准备处理数据的一种方法。 
# 在该教程中，我们使用batch size 大小为1，这意味着我们所要做的就是将句子对中的单词转换为词汇表中的相应索引，并将其提供给模型。

# 但是，如果你想要加速训练或者想要利用GPU并行计算能力，则需要使用小批量（mini-batches）来训练。

# 使用小批量（mini-batches）也意味着我们必须注意批量处理中句子长度的变化。 
# 为了容纳同一批次中不同大小的句子，我们将使我们的批量输入张量大小（max_length，batch_size），
# 其中短于max_length的句子在EOS_token之后进行零填充（zero padded）。

# 如果我们简单地通过将单词转换为索引（indicesFromSentence和零填充（ zero-pad）将我们的英文句子转换为张量，
# 我们的张量将具有大小（batch_size，max_length），并且索引第一维将在所有时间步骤中返回完整序列。 
# 但是，我们需要沿着时间对我们批量数据进行索引并且包括批量数据中所有序列。 
# 因此，我们将输入批处理大小转换为（max_length，batch_size），以便跨第一维的索引返回批处理中所有句子的时间步长。 
# 我们在zeroPadding函数中隐式处理这个转置。

# liujia: CNN中输入的是(batch_size, channel, width, height), 而RNN输入的是(max_length, batch_size, input_dim)
# 即CNN的第一维是batch_size，而RNN的第一维是max_length，第二维是batch_size
# 这样方便按照第一维，即时间步，将数据序列按照batch的方式输入

#   * inputvar函数处理将句子转换为张量的过程，最终创建正确大小的零填充张量。它还返回批处理中每个序列的长度张量（ tensor of lengths)，
#     长度张量稍后将传递给我们的解码器。
#   * outputvar函数执行与inputvar类似的函数，但他不返回长度张量，而是返回二进制mask tensor和最大目标句子长度。
#     二进制mask tensor的大小与输出目标张量的大小相同，但作为PAD_token的每个元素都是0而其他元素都是1。
#   * batch2traindata只需要取一批句子对，并使用上述函数返回输入张量和目标张量。

def indexesFromSentence(voc, sentence):
    # 两个[]是可以通过 + 来合并的。。。
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue=PAD_token):
    # itertools.zip以元素最少的对象为基准，而zip_longest以最长的为基准
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch) #按照此batch(参数l代表此batch的所有Q句子)的Q的句子的最大长度进行padding
    padVar = torch.LongTensor(padList) #将数字的list，转为tensor
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch) #按照此batch(参数l代表此batch的所有A句子)的A的句子的最大长度进行padding
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask) #ByteTensor.....每个元素是个uint8
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs

# 1）liujia: 将整理好的“QA句子对”按照Q句子的长度从大到小排序

# 2）liujia: 并且注意！！！
# 输入的pair_watch是batch_size维度，然后对其中每个QA对(即两个句子)处理后的结果是"最大长度"维度的list，
# 每个元素是句子中的一个单词。并且inp的第一维的最大长度是这个batch的Q句子的最大长度, 而output的第一维的最大长度是这个batch的A句子的最大长度
# 这样处理后，返回的inp和output的维度是(seq_len, batch_size)，而lengths和mask维度是batch_size。当然这个seq_len对应inp和output不一样，
# 并且各个batch也不一样
# 这里要予以重视，因为下面马上要说，torch里RNN模型的输入size是(seq_len, batch_size, input_dim)这里已经准备好前两维了
# 后面对单词index进行embedding即可得到完整的输入

# 输入： QA句子对的list
# 输出：
#  * inp  Q句子做完padding后的"word index"序列的LongTensor序列，即inp是序列，其每个元素是Q句子的每个单词的word index的序列
#  * lengths Q句子的实际长度的tensor序列，每个元素对应Q句子的实际长度
#  * output 对应Q句子的A句子的做完padding后的"word index"序列的LongTensor序列
#  * mask 对应的A句子的mask序列，每个元素是一个1、0序列，1对应Q句子中未padding的单词，0对应padding的单词。句子最后一个为句子结束标志，所以永远是1
#  * max_target_len 对应的A句子的最大长度，是这个batch里的最大长度！
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True) #liujia: 按照问句(维度为0)的长度从大到小排序
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)]) # random.choice(list)从list随机挑选出来一个元素
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("input_variable size:", input_variable.size())
print("lengths:", lengths)
print("lengths size:", lengths.size())
print("target_variable:", target_variable)
print("target_variable size:", target_variable.size())
print("mask:", mask)
print("mask size:", mask.size())
print("max_target_len:", max_target_len)

input_variable: tensor([[ 931,   18, 5438,   28,   27],
        [   5,  128, 6823,  187,   37],
        [1352,    5,   16,  119,    2],
        [ 842,  873,  361,  847,    0],
        [ 144,  128, 6779,   16,    0],
        [  16,  238,   16,    2,    0],
        [ 466,   11,    2,    0,    0],
        [   8,   80,    0,    0,    0],
        [1441, 3148,    0,    0,    0],
        [  41,   16,    0,    0,    0],
        [1302,    2,    0,    0,    0],
        [1991,    0,    0,    0,    0],
        [  37,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
input_variable size: torch.Size([14, 5])
lengths: tensor([14, 11,  7,  6,  3])
lengths size: torch.Size([5])
target_variable: tensor([[  279,    28,  2368,    28,   376],
        [  121,   221,   571,   102, 13005],
        [    2, 12143,   591,   430,   121],
        [    0,     5,    94,    70,   128],
        [    0,   151,    28,    87,   238],
        [    0,    87,    11,    26,    85],
        [    0,    62,  543

In [12]:
# 定义模型

# ---------------------------------------------------------------------------------------
# 这里大概说一下pytorch的RNN的一些用法，参考： https://www.jianshu.com/p/b942e65cb0a3
# Torch中的CNN和RNN中的batchSize的默认位置是不同的。
# CNN中：batchsize的位置是position 0.
# RNN中：batchsize的位置是position 1.

# 在RNN中输入数据格式：
# 对于最简单的RNN，我们可以使用两种方式来调用,torch.nn.RNNCell(),它只接受序列中的单步输入，必须显式的传入隐藏状态。
# torch.nn.RNN()可以接受一个序列的输入，默认会传入一个全0的隐藏状态，也可以自己申明隐藏状态传入。

# 输入大小是三维tensor[seq_len,batch_size,input_dim]
# input_dim是输入的维度，比如是128   通常tokenizer之后embeding的维度
# batch_size是一次往RNN输入句子的数目，比如是5。
# seq_len是一个句子的最大长度，比如15   通常是定义一个最大的长度，不足的部分补一个特殊的token，比如TOKEN_PAD
# 所以千万注意，RNN输入的是序列，一次把批次的所有句子都输入了，得到的ouptut和hidden都是这个批次的所有的输出和隐藏状态，维度也是三维。
# **可以理解为现在一共有batch_size个独立的RNN组件，RNN的输入维度是input_dim，
# 总共输入seq_len个时间步，则每个时间步输入到这个整个RNN模块的维度是[batch_size,input_dim]
# 例如：
# 构造RNN网络，x的维度5，隐层的维度10,网络的层数2
# rnn_seq = nn.RNN(5, 10, 2)  
# 构造一个输入序列，句长为 6，batch 是 3， 每个单词使用长度是 5的向量表示
# x = torch.randn(6, 3, 5) # 6是seq_len，3是batch_size, 5是input_dim和上面的5要对应起来
# out,ht = rnn_seq(x) #h0可以指定或者不指定，或者 out,ht = rnn_seq(x,h0) 指定h0

# 问题1：这里out、ht的size是多少呢？
# 回答：out:6 * 3 * 10, ht: 2 * 3 * 10，
# out的输出维度[seq_len, batch_size, output_dim]，ht的维度[num_layers * num_directions, batch, hidden_size],
# 如果是单向单层的RNN那么一个句子只有一个hidden。num_directions应该是单向的是1，双向的是2
# liujia: 这里的output_dim应该和hidden_size一样吧。。。。见问题2，就是一样的
# liujia: out的输出维度，最后一维应该是out_dim * n_direction?

# 问题2：out[-1]和ht[-1]是否相等？
# 回答：相等，隐藏单元就是输出的最后一个单元，可以想象，每个的输出其实就是那个时间步的隐藏单元

# RNN的其它参数
# RNN(input_dim ,hidden_dim ,num_layers ，…)
# – input_dim 表示输入的特征维度
# – hidden_dim 表示输出的特征维度，如果没有特殊变化，相当于out
# – num_layers 表示网络的层数  liujia:多层相当于stack那种架构
# – nonlinearity 表示选用的非线性激活函数，默认是 ‘tanh’
# – bias 表示是否使用偏置，默认使用  liujia: 这个是什么？输出的out是hidden state经过变换而来的，这个是变换的偏置么？
# – batch_first 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位
# – dropout 表示是否在输出层应用 dropout   liujia: dropout怎么做？
# – bidirectional 表示是否使用双向的 rnn，默认是 False

# LSTM的输出多了一个memory单元
# 例如：
# 输入维度 50，隐层100维，两层
# lstm_seq = nn.LSTM(50, 100, num_layers=2)
# 输入序列seq=10，batch=3，输入维度=50
# lstm_input = torch.randn(10, 3, 50)
# out, (h, c) = lstm_seq(lstm_input) # 使用默认的全0隐藏状态

# 问题1：out和(h,c)的size各是多少？
# 回答：out：(10 * 3 * 100)，(h,c)：都是(2 * 3 * 100) 和上面的传统RNN一样。。。
# 问题2：out[-1,:,:]和h[-1,:,:]相等吗？
# 回答： 相等

# GRU比较像传统的RNN
# gru_seq = nn.GRU(10, 20,2) # x_dim,h_dim,layer_num
# gru_input = torch.randn(3, 32, 10) # seq，batch，x_dim
# out, h = gru_seq(gru_input)

# ---------------------------------------------------------------------------------------

# Seq2Seq模型
# 我们聊天机器人的大脑是序列到序列（seq2seq）模型。 seq2seq模型的目标是将可变长度序列作为输入，
# 并使用“固定大小”的模型将可变长度序列作为输出返回。

# Sutskever et al(https://arxiv.org/abs/1409.3215). 发现通过一起使用两个独立的RNN，我们可以完成这项任务。
# 第一个RNN充当编码器，其将可变长度输入序列编码为固定长度上下文向量。 
# 理论上，该上下文向量（RNN的最终隐藏层）将包含关于输入到机器人的查询语句的语义信息。 
# 第二个RNN是一个解码器，它接收输入文字和上下文矢量，并返回序列中下一句文字的概率和在下一次迭代中使用的隐藏状态。
# 图片： https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_intro/

# liujia: 第一个RNN即编码器，将可变长度的序列编码为一个向量，即语义信息。注意是可变长度，这个解决方法是设置一个最大
# 长度，第第一个RNN的单元数就是这个最大长度，然后将可变序列中比较短的序列的不足的部分，用PAD_TOKEN补足

# liujia: 编码器最后一个hidden layer的输出作为句子的编码输出(如果是双向的就把两个加起来....)
# 不是最后一个hidden state，而是最后一个的输出哦

# 编码器
# 编码器RNN每次迭代中输入一个语句输出一个token（例如，一个单词），同时在这时间内输出“输出”向量和“隐藏状态”向量。 
# 然后将隐藏状态向量传递到下一步，并记录输出向量。编码器将其在序列中的每一点处看到的上下文转换为高维空间中的一系列点，
# 解码器将使用这些点为给定任务生成有意义的输出。

# 我们的编码器的核心是由 Cho et al. 等人发明的多层门循环单元。 在2014年，我们将使用GRU的双向变体，
# 这意味着基本上有两个独立的RNN：一个以正常的顺序输入输入序列，另一个以相反的顺序输入输入序列。 
# 每个网络的输出在每个时间步骤求和。使用双向GRU将为我们提供编码过去和未来上下文的优势。
# 图片： https://colah.github.io/posts/2015-09-NN-Types-FP/

# 注意:embedding层用于在任意大小的特征空间中对我们的单词索引进行编码。 
# 对于我们的模型，此图层会将每个单词映射到大小为hidden_size的特征空间。 
# 训练后，这些值会被编码成和他们相似的有意义词语。

# 最后，如果将填充的一批序列传递给RNN模块，我们必须分别使用torch.nn.utils.rnn.pack_padded_sequence
# 和torch.nn.utils.rnn.pad_packed_sequence在RNN传递时分别进行填充和反填充。

# 计算图:  liujia: 即计算步骤
#   1： 将单词索引转换为词嵌入 embeddings。
#   2： 为RNN模块打包填充批次序列。
#   3： 通过GRU进行前向传播。
#   4： 反填充。
#   5： 对双向GRU输出求和。
#   6： 返回输出和最终隐藏状态。

# liujia: 注意输入的句子已经根据长度(单词数)从大到小排序
# liujia: pack_padded_sequence和pad_packed_sequence，参考：https://www.cnblogs.com/sbj123456789/p/9834018.html
# 主要目的是RNN中对于padding后的序列，不需要对padding部分做编码，而且为了方便输入，要将padding部分“压紧”。
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        print('EncoderRNN::__init__(), enter')
        print('EncoderRNN, n_layers:', n_layers)
        print('EncoderRNN, hidden_size:', hidden_size)
        print('EncoderRNN, dropout:', dropout)
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        # because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, 
                          hidden_size, 
                          n_layers,
                          dropout=(0 if n_layers == 1 else dropout), 
                          bidirectional=True)

    # 输入:
    #   * input_seq：一批输入句子; shape =（max_length，batch_size）
    #   * input_lengths：一批次中每个句子对应的句子长度列表;shape=(batch_size)
    #   * hidden:隐藏状态; shape =(n_layers x num_directions，batch_size，hidden_size)
    # 输出:
    #   * outputs：GRU最后一个隐藏层的输出特征（双向输出之和）; shape =（max_length，batch_size，hidden_size）
    #   * hidden：从GRU更新隐藏状态; shape =（n_layers * num_directions，batch_size，hidden_size）
    
    # liujia: input_seq input_lengths需要安装长度从大到小排列，这个应该是pack_padded_sequence的要求
    def forward(self, input_seq, input_lengths, hidden=None):
        print('EncoderRNN::forward(), enter')
        print('EncoderRNN, input_seq size:', input_seq.size())
        print('EncoderRNN, input_lengths:', input_lengths)
        print('EncoderRNN, input_lengths size:', input_lengths.size())
        print('EncoderRNN, input_lengths sum:', torch.sum(input_lengths))
        print('EncoderRNN, hidden size:', hidden.size() if hidden else "None")
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        print('EncoderRNN, embedded size:', embedded.size())
        # Pack padded batch of sequences for RNN module
        # liujia: 注意经过pack_padded_sequence处理后，tensor就变成PackedSequence对象了，通过.data访问对应的tensor数据，
        #         通过.batch_sizes得到每个时间步长的token数
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        print('EncoderRNN, packed sequence size:', packed.data.size())
        # Forward pass through GRU
        # liujia: 返回的outputs是每个句子的每个单词都有一个，而hidden只是每个句子的最后的那个hidden
        outputs, hidden = self.gru(packed, hidden)
        print('EncoderRNN, after gru, outputs size:', outputs.data.size(), " ,hidden size:", hidden.size())
        # Unpack padding
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        print('EncoderRNN, after unpad, outputs size:', outputs.size())
        # Sum bidirectional GRU outputs.
        # liujia: 这里的“sum”指的是将两个方向的GRU的输出最后一维在数值上加起来
        # 原来的outputs的shape是（max_length，batch_size，2*hidden_size）,加完后变为（max_length，batch_size，hidden_size）
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        print('EncoderRNN, after sum, outputs size:', outputs.size())
        # Return output and final hidden state
        return outputs, hidden

# 输出
# EncoderRNN, input_seq size: torch.Size([14, 64]) #64是batch_size，14是此batch内input_seq的最大长度
# EncoderRNN, input_lengths size: torch.Size([64]) #64是batch_size
# EncoderRNN, hidden size: None
# EncoderRNN, embedded size: torch.Size([14, 64, 500]) #即将input_seq的index进行embedding后，embedding的维度500
# EncoderRNN, packed sequence size: torch.Size([486, 500]) #将上一步的embedded“压紧”，486是input_lengths求和，即此batch所有句子的总的实际长度之和
# EncoderRNN, after gru, outputs size: torch.Size([486, 1000]) ,hidden size: torch.Size([4, 64, 500]) #output的第一维同上，第二维是“双向”gru的输出放在一起了。hidden是因为n_layers为2且双向，所有有4个hidden state
# EncoderRNN, after unpad, outputs size: torch.Size([14, 64, 1000]) #unpack后与input_seq一样第一维是14第二维是64了，但1000仍然是双向两个output, 前500是正向，后500是反向吧。
# EncoderRNN, after sum, outputs size: torch.Size([14, 64, 500]) # 将最后一维的1000，拆成2个500，然后加起来。

In [13]:
# 解码器

# 解码器RNN以token-by-token的方式生成响应语句。它使用编码器的上下文向量和内部隐藏状态来生成序列中的下一个单词。 
# 它持续生成单词，直到输出是EOS_token，这个表示句子的结尾。一个vanilla seq2seq解码器的常见问题是，
# 如果我们只依赖于上下文向量来编码整个输入序列的含义，那么我们很可能会丢失信息。
# 尤其是在处理长输入序列时(所以要引入attention)，这极大地限制了我们的解码器的能力。

# 为了解决这个问题，,Bahdanau et al. 等人创建了一种“attention mechanism”，
# 允许解码器关注输入序列的某些部分，而不是在每一步都使用完全固定的上下文。

# 用解码器的当前隐藏状态和编码器输出来计算注意力。输出注意力的权重与输入序列具有相同的大小，允许我们将它们乘以编码器输出，
# 给出一个加权和，表示要注意的编码器输出部分。 Sean Robertson 的图片很好地描述了这一点：
# https://img.cntofu.com/book/pytorch-doc-zh/docs/1.0/img/603ac943f18d1acfa71487283e63f35f.jpg

# Luong et al. 通过创造“Global attention”，改善了Bahdanau et al. 的基础工作。 
# 关键的区别在于，对于“Global attention”，我们考虑所有编码器的隐藏状态，而不是Bahdanau等人的“Local attention”，
# 它只考虑当前步中编码器的隐藏状态。 另一个区别在于，通过“Global attention”，
# 我们仅使用当前步的解码器的隐藏状态来计算注意力权重（或者能量）。 
# Bahdanau等人的注意力计算需要知道前一步中解码器的状态。 此外，Luong等人提供各种方法来计算编码器输出和解码器输出之间的注意权重（能量），
# 称之为“score functions”：https://github.com/apachecn/pytorch-doc-zh/raw/master/docs/1.0/img/7818f6b40cbd799eddec20743b45fde5.jpg

# 总体而言，Global attention机制可以通过下图进行总结。请注意，我们将“Attention Layer”用一个名为Attn的nn.Module来单独实现。 
# 该模块的输出是经过softmax标准化后权重张量的大小（batch_size，1，max_length）。
# 如图： https://pytorch.org/tutorials/_images/global_attn.png

# liujia:
# attention操作就是根据这个时间步RNN的输出，和之前编码器所有的输出(这个输出的第一维是seq_len)，计算一个weight
# 这个weight是对于编码器所有步输出的一个权重，然后将编码器所有输出安装这个权重加权求和，得到一个向量(维度就是输出向量的维度，这里是500)
# 然后将本时间步RNN的输出和这个向量，拼接到一起，然后经过一个matrix(看做一个全连接层),维度是(2*hidden_size, hidden_size)，得到一个hidden_size的向量(这里是500)
# 然后再接一个tanh非线性变换，得到最终的output向量
# 例如，返回一个[64, 1, 15]向量，64是batch_size，1是表示单步输入的，15是encoder_output,encoder有15个输出，而这里是这些输出的权重
# 这个attention layer只返回权重，而不管后面怎么计算最终的attention context，以及怎么和decoder当前时间步的output融合

# Luong的attention layer
# ht=当前目标解码器状态，hs=所有编码器状态
# 注意里面的torch.nn.Linear(m,n)，相当于一个m*n的矩阵，self.attn(x)，即是将x与此m*n的矩阵相乘( x * 此矩阵)，若x维度为(x,m)则结果为(x,n)
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    # 输出
    # Attn::forward(), enter
    # Attn, method: dot
    # Attn, hidden size: torch.Size([1, 64, 500])
    # Attn, encoder_outputs size: torch.Size([15, 64, 500])
    # Attn, after score, attn_energies size: torch.Size([15, 64])
    def dot_score(self, hidden, encoder_output):
        # dot_score(ht, hs) = <ht, hs>
        # 输出： torch.Size([15, 64, 500])
        # liujia: torch的乘法参考：https://blog.csdn.net/da_kao_la/article/details/87484403
        # 这里的 “*”，即点乘，应该是两个tensor的对应元素相乘
        # 但这里第一维，hidden是1，encoder_output是15，剩下两个维度分别是64和500，所以对维度是“1”的，要做broadcasting，扩展到15
        # 最后效果相当于hidden分别乘以encoder_output的第一维的每一个元素。即相当于拿hidden的batch里每一个向量去“点乘”encoder_output的
        # 15个seq里每一个batch的的向量，然后对这个向量求和，作为加权值
        # 最后返回结果即为(15, 64)
        print('Attn, dot_score score, hidden * encoder_output size:', (hidden * encoder_output).size())
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        # general_score(ht, hs) = ht * W * hs
        energy = self.attn(encoder_output) # 相当于W*hs
        print('Attn, general_score score, energy size:', energy.size())
        return torch.sum(hidden * energy, dim=2) #相当于ht * (W*hs)

    def concat_score(self, hidden, encoder_output):
        # concat_score(ht, hs) = v*tanh(W*[ht;hs]) 所有这里的attn是(self.hidden_size * 2, hidden_size)，两个hidden_size进来，一个hidden_size出去
        # torch.cat(hidden.expand(encoder_output.size(0), -1, -1), encoder_output) 相当于 [ht;hs]
        # self.attn 相当于 W*[ht;hs]，最后的tanh()相当于tanh(W*[ht;hs])
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        print('Attn, concat_score score, energy size:', energy.size())
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        print('Attn::forward(), enter')
        print('Attn, method:', self.method)
        print('Attn, hidden size:', hidden.size()) # [1, 64, 500]
        print('Attn, encoder_outputs size:', encoder_outputs.size()) #[15, 64, 500]
        # 根据给定的方法计算注意力（能量）  
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
        print('Attn, after score, attn_energies size:', attn_energies.size()) # [15, 64]

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()
        print('Attn, after transpose, attn_energies size:', attn_energies.size()) # [64, 15]

        # Return the softmax normalized probability scores (with added dimension)
        sx = F.softmax(attn_energies, dim=1)
        print('Attn, after softmax, size:', sx.size()) # [64, 15] 上面对dim=1做softmax，不改变维度，只不过dim=1这维归一化
        print('Attn, after softmax 2, size:', sx.unsqueeze(1).size()) # [64, 1, 15] 加上一维
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

    
# 现在我们已经定义了注意力子模块，我们可以实现真实的解码器模型。对于解码器，我们将每次手动进行一批次的输入。
# 这意味着我们的词嵌入张量和GRU输出都将具有相同大小（1，batch_size，hidden_size）。

# 计算图
# 1.获取当前输入的词嵌入
# 2.通过单向GRU进行前向传播
# 3.通过2输出的当前GRU计算注意力权重
# 4.将注意力权重乘以编码器输出以获得新的“weighted sum”上下文向量
# 5.使用Luong eq.5连接加权上下文向量和GRU输出
# 6.使用Luong eq.6预测下一个单词（没有softmax）
# 7.返回输出和最终隐藏状态

# 输入
# * input_step：每一步输入序列batch（一个单词）;shape =（1，batch_size）
# * last_hidden：GRU的最终隐藏层;shape =（n_layers x num_directions，batch_size，hidden_size）
# * encoder_outputs：编码器模型的输出;shape =（max_length，batch_size，hidden_size）
# 输出
# * output: 一个softmax标准化后的张量， 代表了每个单词在解码序列中是下一个输出单词的概率;shape =（batch_size，voc.num_words）
# * hidden: GRU的最终隐藏状态;shape =（n_layers x num_directions，batch_size，hidden_size

# liujia: 一些说明
# * nn.Dropout(float ratio), 对每个元素按照ratio的概率，将其置为0。nn.Dropout2d是按照概率将某个通道全置为0
# * encoder_outputs.transpose(0, 1)是将第二维和第一维转置
# * attn_weights.bmm(Tensor)，是batch matrix multiply，即批量矩阵乘法，如维度(10,3,4)和维度(10,4,5)做bmm，结果是(10,3,5)
#   即第一维是batch size，然后后面的东东做矩阵乘法，结果第一维还是这个batch size
# * squeeze(N)是去掉维度为1的那一维，不指定N就是所有维度为1的都去掉，指定N则是指定的第N维维度是1则去掉，否则不去。
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 定义层
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        print('LuongAttnDecoderRNN::forward(), enter')
        print('LuongAttnDecoderRNN, input_step size:', input_step.size())
        print('LuongAttnDecoderRNN, last_hidden size:', last_hidden.size())
        print('LuongAttnDecoderRNN, encoder_outputs size:', encoder_outputs.size())
        # 注意：我们一次运行这一步（单词）
        # 获取当前输入字的嵌入
        embedded = self.embedding(input_step)
        print('LuongAttnDecoderRNN, after embedding, embedded size:', embedded.size())
        embedded = self.embedding_dropout(embedded)
        print('LuongAttnDecoderRNN, after dropout, embedded size:', embedded.size())
        
        # 通过单向GRU转发
        rnn_output, hidden = self.gru(embedded, last_hidden)
        print('LuongAttnDecoderRNN, after gru, rnn_output size:', rnn_output.size())
        print('LuongAttnDecoderRNN, after gru, hidden size:', hidden.size())
        
        # 从当前GRU输出计算注意力
        attn_weights = self.attn(rnn_output, encoder_outputs)
        print('LuongAttnDecoderRNN, attention, attn_weights size:', attn_weights.size())
        
        # 将注意力权重乘以编码器输出以获得新的“加权和”上下文向量
        # attn_weights [64, 1, 15], encoder_outputs [15, 64, 500], 交互第一维第二维后[64, 15, 500]
        # 结果的context是[64, 1, 500]
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        print('LuongAttnDecoderRNN, attention, encoder_outputs.transpose(0, 1) size:', encoder_outputs.transpose(0, 1).size())
        print('LuongAttnDecoderRNN, attention, context size:', context.size())
        
        # 使用Luong的公式五连接加权上下文向量和GRU输出
        rnn_output = rnn_output.squeeze(0)
        print('LuongAttnDecoderRNN, after squeeze, rnn_output size:', rnn_output.size())
        context = context.squeeze(1)
        print('LuongAttnDecoderRNN, after squeeze, context size:', context.size())
        
        # torch.cat将两个tensor在某个dim上拼接起来，例如size(2,3)和(4,3)在dim 0拼接后，变为(6,3)
        # 这里两个输入都是[64, 500]，在dim=1拼接后变为[64, 1000]
        concat_input = torch.cat((rnn_output, context), 1) 
        print('LuongAttnDecoderRNN, after concat, concat_input size:', concat_input.size())
        concat_output = torch.tanh(self.concat(concat_input)) # 相当于 tanh(W*concat_input)
        print('LuongAttnDecoderRNN, after concat, self.concat(concat_input) size:', self.concat(concat_input).size())
        print('LuongAttnDecoderRNN, after concat, concat_output size:', concat_output.size())
        
        # 使用Luong的公式6预测下一个单词
        output = self.out(concat_output) # 相当于W*concat_output，将输出的hidden size的向量转为单词表单词数维度的向量
        print('LuongAttnDecoderRNN, to pred, output size:', output.size())
        output = F.softmax(output, dim=1)
        print('LuongAttnDecoderRNN, softmax, output size:', output.size())
        
        # 返回输出和在最终隐藏状态，output [64, 13153],13153是单词数，hidden [2, 64, 500],hidden接下来还要用
        return output, hidden

# 预测单词输出，就是根据最终得到的output向量，然后接一个全连接层，如上就是一个(hidden_size, words_num)的矩阵，转为新的向量
# 这个新的向量的维度就是单词表的单词个数，然后接softmax即可得到输出
    
# 输出
# LuongAttnDecoderRNN::forward(), enter
# LuongAttnDecoderRNN, input_step size: torch.Size([1, 64]) # 按照时间步单步输入，所以是1
# LuongAttnDecoderRNN, last_hidden size: torch.Size([2, 64, 500]) # 2是两层layer，500是hidden state的dim
# LuongAttnDecoderRNN, encoder_outputs size: torch.Size([15, 64, 500]) # encoder层的输出，15是输入的seq_len，即batch里句子的最大长度

# LuongAttnDecoderRNN, after embedding, embedded size: torch.Size([1, 64, 500]) 
# LuongAttnDecoderRNN, after dropout, embedded size: torch.Size([1, 64, 500]) # dropout应该是随机设置为0

# LuongAttnDecoderRNN, after gru, rnn_output size: torch.Size([1, 64, 500]) # gru的输出，这里是时间单步，所以第一维是1
# LuongAttnDecoderRNN, after gru, hidden size: torch.Size([2, 64, 500]) # 2层layer的hidden state

#得到batch内(对应64)，每一个时间步(对应1)的对应的每一个encoder输出的加权和
# LuongAttnDecoderRNN, attention, attn_weights size: torch.Size([64, 1, 15]) 
# LuongAttnDecoderRNN, attention, encoder_outputs.transpose(0, 1) size: torch.Size([64, 15, 500]) #上面的encoder_outputs交换前两维
# bmm矩阵乘法，[64, 1, 15]*[64, 15, 500]，结果是[64, 1, 500]
# LuongAttnDecoderRNN, attention, context size: torch.Size([64, 1, 500]) 

# LuongAttnDecoderRNN, after squeeze, rnn_output size: torch.Size([64, 500]) #之前的[1, 64, 500]，squeeze(0)后变为[64, 500]
# LuongAttnDecoderRNN, after squeeze, context size: torch.Size([64, 500]) # [64, 1, 500],squeeze(1)后变为此。即batch内每一个样本，根据其output和注意力权重，最后加权生成一个500维的向量

# LuongAttnDecoderRNN, after concat, concat_input size: torch.Size([64, 1000]) # 两个[64,500]，dim=1的concate变为[64,1000]
# 首先经过(2*hidden,hidden)的线性层(最后一维从1000变为500)，此时后再tanh
# LuongAttnDecoderRNN, after concat, self.concat(concat_input) size: torch.Size([64, 500])
# LuongAttnDecoderRNN, after concat, concat_output size: torch.Size([64, 500]) 

# LuongAttnDecoderRNN, to pred, output size: torch.Size([64, 13153]) #out操作将500维的向量转为词典个数维的向量
# LuongAttnDecoderRNN, softmax, output size: torch.Size([64, 13153]) #最后softmax就好了

# 最后输出的output即为[64, 13153]，batch内每个样本输出对应单词变内单词的概率，hidden依然是[2, 64, 500]

In [14]:
# 5.定义训练步骤

# 5.1 Masked 损失
# 由于我们处理的是批量填充序列，因此在计算损失时我们不能简单地考虑张量的所有元素。
# 我们定义maskNLLLoss可以根据解码器的输出张量、描述目标张量填充的binary mask张量来计算损失。
# 该损失函数计算与mask tensor中的1对应的元素的平均负对数似然。
                          
# liujia
# torch.gather(input, dim, index) 参考：https://www.jianshu.com/p/5d1f8cd5fe31
# 基本上意思就是在指定dim上，按照index指定的“索引”，获取input相应dim上该“索引”位置的值，放置到返回的tensor的相应位置上
# torch.gather(inp, 1, target.view(-1, 1)) 这里inp [64, 13153]，target.view(-1, 1)变为[64,1], 保存真实单词的索引
# 所以意思是，根据target.view(-1, 1) dim=1处保存的真实单词的索引，从input [64, 13153], dim=1的指定位置，取出对应值，这个值
# 就是真实单词对应的预测概率值(softmax之后)，返回的值即为[64, 1]，即为batch里每个真实单词对应的预测概率值
# 然后求 -log后，去掉dim=1的维度， 不过我感觉去掉dim=1维度再做-log似乎也可以吧
# 这样就得到交叉熵，然后用masked_select(mask)去掉mask=0的，只保留mask=1的，最后求平均即是平均的交叉熵loss
def maskNLLLoss(inp, target, mask):
    print('maskNLLLoss, enter')
    print('maskNLLLoss, inp size:', inp.size()) # [64, 13153] 64是batch_size，13153是单词个数，代表每个单词的softmax归一化后的概率
    print('maskNLLLoss, target size:', target.size()) # [64], 64是batch_size，每个元素代表一个word index，未embedding
    print('maskNLLLoss, mask size:', mask.size()) # [64]，64是batch_size，值是1、0，表明是单词或者padding
    nTotal = mask.sum() # 输入里有多少个是有效的，因为1是正常单词，0是padding，所以统计1的个数就好了
    
    print('maskNLLLoss, target.view(-1, 1) size:', target.view(-1, 1).size()) # [64, 1],将原来[64]维度通过.view(-1,1)变为两维的，且第二维是1
    print('maskNLLLoss, torch.gather(inp, 1, target.view(-1, 1)) size:', torch.gather(inp, 1, target.view(-1, 1)).size())
    print('maskNLLLoss, torch.gather(inp, 1, target.view(-1, 1)).squeeze(1) size:', torch.gather(inp, 1, target.view(-1, 1)).squeeze(1).size() )
    
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    print('maskNLLLoss, crossEntropy size:', crossEntropy.size()) # [64]
    loss = crossEntropy.masked_select(mask).mean()
    print('maskNLLLoss, loss size:', loss.size()) # 这里loss已经是数值了
    loss = loss.to(device)
    return loss, nTotal.item() # 返回的loss是求平均之后的，nTotal是里面有效的个数

# 5.2 单次训练迭代
# train函数包含单次训练迭代的算法（单批输入）。

# 我们将使用一些巧妙的技巧来帮助融合：
# * 第一个技巧是使用teacher forcing。 这意味着在一些概率是由teacher_forcing_ratio设置，
#   我们使用当前目标单词作为解码器 的下一个输入，而不是使用解码器的当前推测。该技巧充当解码器的training wheels，有助于更有效的训练。
#   然而，teacher forcing 可能导致推导中的模型不稳定，因为解码器可能没有足够的机会在训练期间真正地制作自己的输出序列。
#   因此，我们必须注意我们如何设置teacher_forcing_ratio， 同时不要被快速的收敛所迷惑。
# * 第二个技巧是梯度裁剪(gradient clipping)。这是一种用于对抗“爆炸梯度（exploding gradient）”问题的常用技术。
#   本质上，通过将梯度剪切或阈值化到最大值，我们可以防止在损失函数中梯度以指数方式增长并发生溢出（NaN）或者越过梯度。

# 图片： https://github.com/apachecn/pytorch-doc-zh/raw/master/docs/1.0/img/35f76328fb2b83228804b30cf4978e40.jpg
# 图片来源: Goodfellow et al. Deep Learning. 2016. https://www.deeplearningbook.org/

# 操作顺序
# 1.通过编码器前向计算整个批次输入。
# 2.将解码器输入初始化为SOS_token，将隐藏状态初始化为编码器的最终隐藏状态。 # liujia:注意这个！！！
# 3.通过解码器一次一步地前向计算输入一批序列。
# 4.如果是teacher forcing算法：将下一个解码器输入设置为当前目标;如果是no teacher forcing算法：将下一个解码器输入设置为当前解码器输出。
# 5.计算并累积损失。
# 6.执行反向传播。
# 7.裁剪梯度。
# 8.更新编码器和解码器模型参数。

# 说明：
# PyTorch的RNN模块（RNN，LSTM，GRU）可以像任何其他非重复层一样使用，只需将整个输入序列（或一批序列）传递给它们。 
# 我们在编码器中使用GRU层就是这样的。实际情况是，在计算中有一个迭代过程循环计算隐藏状态的每一步。
# 或者，你每次只运行一个 模块。在这种情况下，我们在训练过程中手动循环遍历序列就像我们必须为解码器模型做的那样。
# 只要你正确的维护这些模型的模块，就可以非常简单的实现顺序模型。
# liujia: 就是说torch中的RNN模块可以批量输入，也可以手工控制按照时间步一步步的输入并记录输出和隐藏状态

# 这里对一个batch的所有时间步的数据进行训练
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):
    
    print('train, enter')
    print('train, input_variable size:', input_variable.size())
    print('train, lengths:', lengths.size())
    print('train, target_variable size:', target_variable.size())
    print('train, mask size:', mask.size())
    print('train, max_target_len:', max_target_len)
    print('train, batch_size:', batch_size)

    # 零化梯度
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # 设置设备选项
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # 初始化变量
    loss = 0
    print_losses = []
    n_totals = 0

    # 正向传递编码器
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    # liujia: 参考EncoderRNN的forward()函数
    # 前两维与input_variable一样，分别是此batch输入句子的最大长度和batch_size，最后500是处理过的，将两个500合并为一个了
    print('train, after encoder, encoder_outputs size:', encoder_outputs.size())
    # 第一维是4，因为是双向且两层网络，第二维是batch_size64, 最后一维是设置的hidden state的dim
    print('train, after encoder, encoder_hidden size:', encoder_hidden.size())

    # 创建初始解码器输入（从每个句子的SOS令牌开始）
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    print('train, after encoder, origin decoder_input size:', decoder_input.size())

    # 将初始解码器隐藏状态设置为编码器的最终隐藏状态  liujia: 这个要注意！
    # encoder_hidden前两个应该对应正向的两层layer的hidden state，后面两个对应反向的两层layer的hidden state
    # 注意encoder是双向GRU，而decoder是单向GRU
    decoder_hidden = encoder_hidden[:decoder.n_layers] 
    print('train, set decoder hidden state, decoder.n_layers:', decoder.n_layers) # 2层
    print('train, set decoder hidden state, decoder_hidden size:', decoder_hidden.size()) #[2, 64, 500]，取encoder_hidden的前两个

    # 确定我们是否此次迭代使用`teacher forcing`
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # 通过解码器一次一步地转发一批序列
    # liujia: decoder接受三个参数
    #  * decoder_input: 输入向量(是单词的index，未embedding)，(1,64), 1是因为这里是按照时间步输入，64是batch_size。第一步都是句子开始SOS_TOKEN
    #  * decoder_hidden: 输入的隐藏状态, (2, 64, 500)，2是因为这是两层，64是batch_size，500是hidden state的维度
    #  * encoder_outputs:
    if use_teacher_forcing:
        for t in range(max_target_len):
            # 输入：decoder_input [1, 64] 未做embedding，decoder_hidden [2, 64, 500]，encoder_outputs [15, 64, 500]
            # 输出：decoder_output [64, 13153]，decoder_hidden [2, 64, 500]
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # Teacher forcing: 下一个输入是当前的目标(即是真实的输入)
            decoder_input = target_variable[t].view(1, -1)
            print('train, get input, target_variable[t] size:', target_variable[t].size()) # [64]
            # [1, 64]，view(1,-1)意思是将第一维设为1，第二维根据情况自动设置。还是需要转为标准格式，1是此时间步且单步，64是batch_size
            print('train, get input, target_variable[t].view(1, -1) size:', decoder_input.size()) 
            
            # 计算并累计损失
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal) # 加上这个时间步的总的loss
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # No teacher forcing: 下一个输入是解码器自己的当前输出
            _, topi = decoder_output.topk(1) # 我猜topi是[64,1],保存对应最大值的index，即最大值对应的word index
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]]) #生成decoder_input
            decoder_input = decoder_input.to(device)
            
            # 计算并累计损失
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # 执行反向传播
    loss.backward()

    # 剪辑梯度：梯度被修改到位
    # liujia: 为什么先剪辑梯度，再更新模型梯度？
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # 调整模型权重
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

# 5.3 训练迭代
# 现在终于将完整的训练步骤与数据结合在一起了。给定传递的模型、优化器、数据等，trainIters函数负责运行n_iterations的训练。 
# 这个功能显而易见，因为我们通过train函数的完成了繁重工作。

# 需要注意的一点是，当我们保存模型时，我们会保存一个包含编码器和解码器state_dicts（参数）、优化器的state_dicts、损失、迭代等的压缩包。
# 以这种方式保存模型将为我们checkpoint提供最大的灵活性。加载checkpoint后，我们将能够使用模型参数进行推理，
# 或者我们可以在我们中断的地方继续训练。
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 为每次迭代加载batches
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # 初始化
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # 训练循环
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # 从batch中提取字段
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # 使用batch运行训练迭代
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # 打印进度
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # 保存checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))
            

In [15]:
# 6.评估定义

# 在训练模型后，我们希望能够自己与机器人交谈。首先，我们必须定义我们希望模型如何解码编码输入。

# 6.1 贪婪解码
# 贪婪解码是我们在不使用teacher forcing时在训练期间使用的解码方法。换句话说，对于每一步，
# 我们只需从具有最高softmax值的decoder_output中选择单词。该解码方法在单步长级别上是最佳的。

# 为了便于贪婪解码操作，我们定义了一个GreedySearchDecoder类。当运行时，
# 类的实例化对象输入序列（input_seq）的大小是（input_seq length，1）， 
# 标量输入（input_length）长度的张量和 max_length 来约束响应句子长度。使用以下计算图来评估输入句子：
# 计算图
# 1.通过编码器模型前向计算。
# 2.准备编码器的最终隐藏层，作为解码器的第一个隐藏输入。
# 3.将解码器的第一个输入初始化为 SOS_token。
# 4.将初始化张量追加到解码后的单词中。
# 5.一次迭代解码一个单词token：
#   (i)通过解码器进行前向计算。
#   (ii)获得最可能的单词token及其softmax分数。
#   (iii)记录token和分数。
#   (iv)准备当前token作为下一个解码器的输入。
# 6.返回收集到的词 tokens 和分数。

class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # 通过编码器模型转发输入
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # 准备编码器的最终隐藏层作为解码器的第一个隐藏输入
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # 使用SOS_token初始化解码器输入
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # 初始化张量以将解码后的单词附加到
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # 一次迭代地解码一个词tokens
        for _ in range(max_length):
            # 正向通过解码器
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # 获得最可能的单词标记及其softmax分数
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # 记录token和分数
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # 准备当前令牌作为下一个解码器输入（添加维度）
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # 返回收集到的词tokens和分数
        return all_tokens, all_scores
    
# 6.2 评估我们的文本
# 现在我们已经定义了解码方法，我们可以编写用于评估字符串输入句子的函数。
# evaluate函数管理输入句子的低层级处理过程。我们首先使 用batch_size == 1将句子格式化为输入batch的单词索引。
# 我们通过将句子的单词转换为相应的索引，并通过转换维度来为我们的模型准备 张量。
# 我们还创建了一个lengths张量，其中包含输入句子的长度。在这种情况下，lengths是标量，因为我们一次只评估一个句子（batch_size == 1）。 
# 接下来，我们使用我们的GreedySearchDecoder实例化后的对象（searcher）获得解码响应句子的张量。
# 最后，我们将响应的索引转换为单 词并返回已解码单词的列表。

# evaluateInput充当聊天机器人的用户接口。调用时，将生成一个输入文本字段，我们可以在其中输入查询语句。
# 在输入我们的输入句子并
# 按Enter后，我们的文本以与训练数据相同的方式标准化，并最终被输入到评估函数以获得解码的输出句子。
# 我们循环这个过程，这样我们可 以继续与我们的机器人聊天直到我们输入“q”或“quit”。

# 最后，如果输入的句子包含一个不在词汇表中的单词，我们会通过打印错误消息并提示用户输入另一个句子来优雅地处理。
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### 格式化输入句子作为batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # 创建lengths张量
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # 转置batch的维度以匹配模型的期望
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # 使用合适的设备
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # 用searcher解码句子
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 获取输入句子
            input_sentence = input('> ')
            # 检查是否退出
            if input_sentence == 'q' or input_sentence == 'quit': break
            # 规范化句子
            input_sentence = normalizeString(input_sentence)
            # 评估句子
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 格式化和打印回复句
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")


In [16]:
# 7.运行模型

# 最后，是时候运行我们的模型了！

# 无论我们是否想要训练或测试聊天机器人模型，我们都必须初始化各个编码器和解码器模型。
# 在接下来的部分，我们设置所需要的配置，选择从头开始或设置检查点以从中加载，并构建和初始化模型。您可以随意使用不同的配置来优化性能。

# 配置模型
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500

# 编解码器都是2层，但是编码器是双向，而解码器是单向(加上attention机制)
encoder_n_layers = 2 
decoder_n_layers = 2

dropout = 0.1
batch_size = 64

# 设置检查点以加载; 如果从头开始，则设置为None
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))

# 如果提供了loadFilename，则加载模型
if loadFilename:
    # 如果在同一台机器上加载，则对模型进行训练
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']

print('Building encoder and decoder ...')
# 初始化词向量
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
    
# 初始化编码器 & 解码器模型
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
    
# 使用合适的设备
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')


Building encoder and decoder ...
EncoderRNN::__init__(), enter
EncoderRNN, n_layers: 2
EncoderRNN, hidden_size: 500
EncoderRNN, dropout: 0.1
Models built and ready to go!


In [17]:
# liujia: 这里就是测试，用于测试上面的各个模块和函数的功能

# 配置训练/优化
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 1
print_every = 1
save_every = 500

# 确保dropout layers在训练模型中
encoder.train()
decoder.train()

# 初始化优化器
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
#if loadFilename:
#    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
#    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

def testIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 为每次迭代加载batches
    # liujia: 对每次n_iteration，调用一次batch2TrainData()函数，此函数返回此iter的数据，数据为一个tuple，有5个元素，这5个元素都是list
    # batch2TrainData()的第二个参数，是一个list其每个元素是一个QA对，是根据batch_size从原始数据随机采样而来
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    
    
    print('training_batches length:', len(training_batches)) #与n_iteration一样，即迭代多少次。
    print('type of elem in training_batches:', type(training_batches[0])) # tuple....
    # 01234分别是inp, lengths, output, mask, max_target_len，参考batch2TrainData函数
    print(training_batches[0][0].size()) # liujia： (q_max_len, batch_size)，即batch2TrainData()返回的第一个值inp，q_max_len此batch里Q句子最大长度
    print(training_batches[0][1].size()) # liujia： (batch_size)，即batch2TrainData()返回的第二个值lengths，代表对应的Q句子的真实长度
    print(training_batches[0][2].size()) # liujia： (a_max_len, batch_size)，即batch2TrainData()返回的第三个值output，a_max_len此batch里A句子最大长度
    print(training_batches[0][3].size()) # liujia： (batch_size)，即batch2TrainData()返回的第四个值mask, A句子里padding部分为1，否则为0
    print(training_batches[0][4]) # liujia: 此batch里A句子的最大长度
    
    # 初始化
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # 训练循环
    print("Testing...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # 从batch中提取字段
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # 使用batch运行训练迭代
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # 打印进度
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # 保存checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))
            
# 运行训练迭代
print("Starting Testing!")
testIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Testing!
training_batches length: 1
type of elem in training_batches: <class 'tuple'>
torch.Size([15, 64])
torch.Size([64])
torch.Size([13, 64])
torch.Size([13, 64])
13
Initializing ...
Testing...
train, enter
train, input_variable size: torch.Size([15, 64])
train, lengths: torch.Size([64])
train, target_variable size: torch.Size([13, 64])
train, mask size: torch.Size([13, 64])
train, max_target_len: 13
train, batch_size: 64
EncoderRNN::forward(), enter
EncoderRNN, input_seq size: torch.Size([15, 64])
EncoderRNN, input_lengths: tensor([15, 15, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 12, 12, 12, 12, 12,
        11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10,  9,  9,  9,  9,  8,  8,  8,
         8,  8,  8,  7,  7,  7,  7,  6,  6,  6,  6,  6,  6,  6,  6,  5,  5,  5,
         5,  5,  5,  4,  4,  4,  4,  3,  3,  3])
EncoderRNN, input_lengths size: torch.Size([64])
EncoderRNN, input_lengths sum: tensor(569)
EncoderRNN, hidden size: None
EncoderRNN, embedded s

../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/sr

 torch.Size([64, 500])
LuongAttnDecoderRNN, after squeeze, context size: torch.Size([64, 500])
LuongAttnDecoderRNN, after concat, concat_input size: torch.Size([64, 1000])
LuongAttnDecoderRNN, after concat, self.concat(concat_input) size: torch.Size([64, 500])
LuongAttnDecoderRNN, after concat, concat_output size: torch.Size([64, 500])
LuongAttnDecoderRNN, to pred, output size: torch.Size([64, 13153])
LuongAttnDecoderRNN, softmax, output size: torch.Size([64, 13153])
train, get input, target_variable[t] size: torch.Size([64])
train, get input, target_variable[t].view(1, -1) size: torch.Size([1, 64])
maskNLLLoss, enter
maskNLLLoss, inp size: torch.Size([64, 13153])
maskNLLLoss, target size: torch.Size([64])
maskNLLLoss, mask size: torch.Size([64])
maskNLLLoss, target.view(-1, 1) size: torch.Size([64, 1])
maskNLLLoss, torch.gather(inp, 1, torch.gather(inp, 1, target.view(-1, 1)) size: torch.Size([64, 1])
maskNLLLoss, torch.gather(inp, 1, target.view(-1, 1)).squeeze(1) size: torch.Size([6

../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/sr


LuongAttnDecoderRNN, after squeeze, context size: torch.Size([64, 500])
LuongAttnDecoderRNN, after concat, concat_input size: torch.Size([64, 1000])
LuongAttnDecoderRNN, after concat, self.concat(concat_input) size: torch.Size([64, 500])
LuongAttnDecoderRNN, after concat, concat_output size: torch.Size([64, 500])
LuongAttnDecoderRNN, to pred, output size: torch.Size([64, 13153])
LuongAttnDecoderRNN, softmax, output size: torch.Size([64, 13153])
train, get input, target_variable[t] size: torch.Size([64])
train, get input, target_variable[t].view(1, -1) size: torch.Size([1, 64])
maskNLLLoss, enter
maskNLLLoss, inp size: torch.Size([64, 13153])
maskNLLLoss, target size: torch.Size([64])
maskNLLLoss, mask size: torch.Size([64])
maskNLLLoss, target.view(-1, 1) size: torch.Size([64, 1])
maskNLLLoss, torch.gather(inp, 1, torch.gather(inp, 1, target.view(-1, 1)) size: torch.Size([64, 1])
maskNLLLoss, torch.gather(inp, 1, target.view(-1, 1)).squeeze(1) size: torch.Size([64])
maskNLLLoss, cross

../aten/src/ATen/native/LegacyDefinitions.cpp:38: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:38: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:38: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:38: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:38: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


Iteration: 1; Percent complete: 100.0%; Average loss: 9.4830


In [61]:
# 7.1 执行训练
# 如果要训练模型，请运行以下部分。

# 首先我们设置训练参数，然后初始化我们的优化器，最后我们调用trainIters函数来运行我们的训练迭代。

# 配置训练/优化
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# 确保dropout layers在训练模型中
encoder.train()
decoder.train()

# 初始化优化器
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# 运行训练迭代
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
EncoderRNN::forward(), enter
EncoderRNN, input_seq size: torch.Size([15, 64])
EncoderRNN, input_lengths size: torch.Size([64])
EncoderRNN, hidden size: None
EncoderRNN, embedded size: torch.Size([15, 64, 500])
EncoderRNN, packed sequence size: torch.Size([523, 500])
EncoderRNN, after gru, outputs size: torch.Size([523, 1000])  ,hidden size: torch.Size([4, 64, 500])
EncoderRNN, after unpad, outputs size: torch.Size([15, 64, 1000])
EncoderRNN, after sum, outputs size: torch.Size([15, 64, 500])


../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:48: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/sr

KeyboardInterrupt: 